In [5]:
# !pip install simpletransformers

In [6]:
import json
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
with open(r"train-v2.0.json", "r") as read_file:
    data = json.load(read_file)

## Chuẩn bị custom dataset

In [8]:
def transform_json(input_json,test_size=30):
    transformed_train = []
    transformed_test = []
    n=0

    for item in input_json['data']:
        for paragraph in item['paragraphs']:
            transformed_item = {
                "context": paragraph.get('context', ''),
                "qas": []
            }
            for qas in paragraph['qas']:
                transformed_qas = {
                    "id": qas['id'],
                    "is_impossible": qas['is_impossible'],
                    "question": qas['question'],
                    "answers": []
                }
                for answer in qas['answers']:
                    transformed_answer = {
                        "text": answer['text'],
                        "answer_start": answer['answer_start']
                    }
                    transformed_qas['answers'].append(transformed_answer)
                transformed_item['qas'].append(transformed_qas)
            n+=1
            if n < test_size:
              transformed_test.append(transformed_item)
            else:
              transformed_train.append(transformed_item)

    return transformed_test, transformed_train

transformed_test_json, transformed_train_json = transform_json(data)

with open('/content/train.json', 'w') as f:
    json.dump(transformed_train_json, f, indent=4)
with open('/content/test.json', 'w') as f:
    json.dump(transformed_test_json, f, indent=4)

In [9]:
with open(r"train.json", "r") as read_file:
    train = json.load(read_file)
with open(r"test.json", "r") as read_file:
    test = json.load(read_file)

## Chọn model

In [10]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [11]:
# model_args = QuestionAnsweringArgs()
# model_args.train_batch_size = 64
# model_args.evaluate_during_training = True
# model_args.n_best_size=3
# model_args.num_train_epochs=5

## Cấu hình


In [12]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 2000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "evaluate_during_training": True,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    "n_gpu": 1,
    # "manual_seed": 4,
    "use_multiprocessing": True,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [13]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

## Training

In [14]:
model.train_model(train, eval_data=test)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 129959/129959 [00:00<00:00, 574758.91it/s]


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


...

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

Running Epoch 5 of 5:   0%|          | 0/1026 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

(5130,
 {'global_step': [1000, 1026, 2000, 2052, 3000, 3078, 4000, 4104, 5000, 5130],
  'correct': [158, 156, 167, 163, 169, 168, 173, 167, 168, 168],
  'similar': [157, 164, 164, 170, 150, 160, 152, 156, 155, 157],
  'incorrect': [45, 40, 29, 27, 41, 32, 35, 37, 37, 35],
  'train_loss': [1.1158356666564941,
   1.236410140991211,
   0.818427562713623,
   0.7467596530914307,
   0.5440172553062439,
   0.617763876914978,
   0.508910059928894,
   0.3498800992965698,
   0.37017154693603516,
   0.439817875623703],
  'eval_loss': [-5.535807291666667,
   -6.021484375,
   -6.853515625,
   -6.889322916666667,
   -7.471354166666667,
   -7.623697916666667,
   -7.847005208333333,
   -7.96484375,
   -8.251302083333334,
   -8.223958333333334]})

## Lưu và tải model đã huấn luyện

In [15]:
import torch

In [20]:
torch.save(model, '/content/drive/MyDrive/QAmodel.bin')

In [18]:
text_m = torch.load('/content/QAmodel.bin')

## Testing

In [23]:
to_predict = [
    {
        "context": "Tran Duc Thuan is the person who created me, he is 23 years old and graduated from HUIT university, he is working at KSI company.",
        "qas": [
            {
                "question": "where is tran duc thuan work?",
                "id": "0",
            }
        ],
    }
]
answers, probabilities = text_m.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 3530.56it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['KSI company', 'KSI company.', 'KSI']}]
